In [35]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Project.toml`


In [36]:
Pkg.add("ForwardDiff")

   Resolving package versions...
  No Changes to `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Project.toml`
  No Changes to `~/airlab/mpc_ws/src/subcanopy_flight/notebooks/Manifest.toml`


In [37]:
using LinearAlgebra
using ForwardDiff
using BlockDiagonals
using ControlSystems

In [38]:
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
@show T
H = [zeros(1,3); I]
@show H'
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

T = [1.0 0.0 0.0 0.0; 0.0 -1.0 0.0 0.0; 0.0 0.0 -1.0 0.0; 0.0 0.0 0.0 -1.0]
H' = [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]


qtorp (generic function with 1 method)

In [39]:
#Quadrotor parameters
m = 0.73
ℓ = 0.17
J = Diagonal([0.007, 0.007, 0.012])
g = 9.81
# kt=1.0
# km=0.0245

h = 0.01 #100 Hz

Nx = 10     # number of states (quaternion)
Nx̃ = 9     # number of states (linearized)
Nu = 4     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [40]:
function E(q)
    E = BlockDiagonal([1.0*I(3), G(q), 1.0*I(3)])
end

E (generic function with 1 method)

In [41]:
# function quad_dynamics(x,u)
#     r = x[1:3]
#     q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
#     v = x[8:10]
#     ω = x[11:13]
#     Q = qtoQ(q)
    
#     ṙ = Q*v
#     q̇ = 0.5*L(q)*H*ω
    
#     v̇ = Q'*[0; 0; -g] + (1/m)*[zeros(2,4); kt*ones(1,4)]*u - hat(ω)*v
    
#     ω̇ = J\(-hat(ω)*J*ω + [0 ℓ*kt 0 -ℓ*kt; -ℓ*kt 0 ℓ*kt 0; km -km km -km]*u)
    
#     return [ṙ; q̇; v̇; ω̇]
# end

# New quadrotor dynamics with control inputs as [F; wx; wy; wz]
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = u[2:4]
    Q = qtoQ(q)
    
    ṙ = Q*v
    q̇ = 0.5*L(q)*H*ω
    
    v̇ = [0; 0; -g] + Q*[0; 0; u[1]]
        
    return [ṙ; q̇; v̇]
end

quad_dynamics (generic function with 1 method)

In [42]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [43]:
#Initial Conditions
uhover = [g; 0; 0; 0]
r0 = [0.0; 0; 1.0]
q0 = [1.0; 0; 0; 0]
v0 = zeros(3)
x0 = [r0; q0; v0];

In [44]:
#Linearize dynamics about hover
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),x0)
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x0,u),uhover)

10×4 Matrix{Float64}:
 0.0      0.0        1.635e-6   0.0
 0.0     -1.635e-6   0.0        0.0
 5.0e-5   0.0        0.0        0.0
 0.0      0.0        0.0        0.0
 0.0      0.005      0.0        0.0
 0.0      0.0        0.005      0.0
 0.0      0.0        0.0        0.005
 0.0      0.0        0.0004905  0.0
 0.0     -0.0004905  0.0        0.0
 0.01     0.0        0.0        0.0

In [45]:
rank(A)#The naive A matrix will always be rank deficient due to the unit-norm constraint on the quaternion

9

In [46]:
#Controlability Matrix
C = B
for k = 1:(Nx-1)
    C = [C A*C[:,end-(Nu-1):end]]
end

In [47]:
rank(C) #The naive linearized system will always be uncontrollable

9

In [48]:
@show size(A)
@show size(B)

size(A) = (10, 10)
size(B) = (10, 4)


(10, 4)

In [49]:
#Reduced system
Ã = Array(E(q0)'*A*E(q0))
B̃ = Array(E(q0)'*B);

@show size(Ã)
@show size(B̃)

size(Ã) = (9, 9)
size(B̃) = (9, 4)


(9, 4)

In [50]:
#Controlability Matrix
C = B̃
for k = 1:(Nx-1)
    C = [C Ã*C[:,end-(Nu-1):end]]
end

In [51]:
rank(C) #Reduced system is controllable

9

In [62]:
# Cost weights
Q = Array(I(Nx̃));
R = Array(I(Nu));

In [63]:
#LQR Controller
K = dlqr(Ã,B̃,Q,R)

@show K

K = [-1.0667140985557689e-13 -2.175315493855221e-12 0.9913771737165269 3.953269603753079e-11 -1.662128405610177e-12 7.107619135214316e-12 -2.737897191255594e-13 -2.5207753229156137e-12 1.722086829401962; 2.1720599295699934e-12 -0.9736067886335726 -7.46056555631606e-12 10.55622672869207 1.3458789638320923e-11 -3.5860222157534792e-12 2.082750638621178e-12 -1.412649020039446 -8.277821025445705e-13; 0.9736067886168867 7.758238496080594e-13 -1.2296401383377256e-12 5.0515421433634335e-12 10.556226728662482 -3.2211860012627777e-13 1.4126490200323105 3.608224830031759e-14 -9.737249213706758e-13; 2.2330331413022383e-13 -1.1314006243125785e-12 -9.790580910661936e-15 1.4136611148354636e-12 3.650994252100662e-12 0.9975031249938582 2.2886913122083675e-13 -2.987440804226478e-13 -1.1154975781278061e-13]


4×9 Matrix{Float64}:
 -1.06671e-13  -2.17532e-12   0.991377     …  -2.52078e-12   1.72209
  2.17206e-12  -0.973607     -7.46057e-12     -1.41265      -8.27782e-13
  0.973607      7.75824e-13  -1.22964e-12      3.60822e-14  -9.73725e-13
  2.23303e-13  -1.1314e-12   -9.79058e-15     -2.98744e-13  -1.1155e-13

In [54]:
#Feedback controller
function controller(x)
    
    q0 = x0[4:7]
    q = x[4:7]
    ϕ = qtorp(L(q0)'*q)
    
    Δx̃ = [x[1:3]-r0; ϕ; x[8:10]-v0]
    
    u = uhover - K*Δx̃
end

controller (generic function with 1 method)

In [55]:
#Simulation
uhist = zeros(Nu,Nt)
xhist = zeros(Nx+3,Nt)
xhist[1:10,1] = [r0+randn(3); L(q0)*rptoq([0.2; -0.9; 0.2]); v0]
for k = 1:(Nt-1)
    uhist[:,k] = controller(xhist[1:10,k])
    xhist[1:10,k+1] = quad_dynamics_rk4(xhist[1:10,k],uhist[:,k])
    xhist[10:12,k] = uhist[2:4,k]
end

In [56]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

In [57]:
model = Quadrotor()
TrajOptPlots.set_mesh!(vis, model)

false

In [58]:
X1 = [SVector{13}(x) for x in eachcol(xhist)];
visualize!(vis, model, thist[end], X1)